In [1]:
import requests
import json
import pandas as pd

# Need to Recapture San Francisco Data
## CO06075 = SF

In [2]:
# New_APIS
attom_keys = ['1aca7ca361664d5b70c603ebd9daa6c7','57fb3873cfd2b0e57cd38d92450c81d1','123b348cfa3af52b50aac55d5fc2a853','81e1f74972e2f35a610c43b670d54de','97a972418c6f6329d2ec3d5fe4e4c31c']

In [3]:
# For each county, we'll make 1 API call and generated 1 text file containing sales data  
# county_geoids is the LIST of all 74 counties in California

# following function pulls all sales data between 200k and 5 million dollars from start of covid
# function takes as input the geoID of the county 
# function outputs a generated text file containing JSON of all SALES transactions between $0.2M and $5M in that county
# function has defaults of from date of 1st Jan 2019 upto 1st Apr 2021. These can be changed as desired.
# i've set pagesize to 50,000 records for each county.

def salesdata_scraper_bycounty_v2(county_geoid,from_date ='2019-01-01', to_date ='2021-04-01'): 
    baseurl = 'https://api.gateway.attomdata.com/propertyapi/v1.0.0/sale/snapshot'
    queryparams = {'geoid':county_geoid, 
               'minsaleamt': 200000,
               'maxsaleamt': 50000000, 
              'startsalesearchdate':from_date,
               'endsalesearchdate': to_date,
                'pagesize' :50000}        
    headers = {'apikey': '97a972418c6f6329d2ec3d5fe4e4c31c', 'accept': 'application/JSON'}
    response = requests.get(baseurl, params = queryparams,headers=headers)
    pydata = response.json()         # converting response object to python list of dictionaries
    number_of_records = pydata['status']['total']
    if number_of_records > 0:   # this condition is to avoid writing empty files in case of zero transactions in a county
        if number_of_records == 10000:
            print('{} county has over 10000 records. Truncated.Run query separately by month instead of year'.format(county_geoid))
        else:    # no files will be created right now for counti 10,000 records
            print(f"Number of records is {number_of_records}")     
            pydata = pydata['property']   # this is to reach inner nested part where our results lie
            with open(county_geoid+'_'+from_date+'-'+to_date, 'w') as outfile:   # The name of each file will be CO06001, CO06003, etc.74 files created            
                json.dump(pydata, outfile) 

In [5]:
time_tuples = [('2019-01-01','2019-03-01'),('2019-03-02','2019-06-01'),('2019-06-02','2019-10-02'),('2019-10-02','2020-02-02'),('2020-02-03','2020-06-03'),('2020-06-04','2020-10-04'),('2020-10-05','2021-02-05'),('2021-02-06','2021-04-01')]

In [7]:
for time in time_tuples:
    start, stop = time
    print(f"Trying {'CO06075'} from {start} to {stop}")
    salesdata_scraper_bycounty_v2(county_geoid = 'CO06075',from_date =start, to_date =stop)
print('done')

Trying CO06075 from 2019-01-01 to 2019-03-01
Number of records is 608
Trying CO06075 from 2019-03-02 to 2019-06-01
Number of records is 1311
Trying CO06075 from 2019-06-02 to 2019-10-02
Number of records is 1877
Trying CO06075 from 2019-10-02 to 2020-02-02
Number of records is 1870
Trying CO06075 from 2020-02-03 to 2020-06-03
Number of records is 1493
Trying CO06075 from 2020-06-04 to 2020-10-04
Number of records is 2088
Trying CO06075 from 2020-10-05 to 2021-02-05
Number of records is 2537
Trying CO06075 from 2021-02-06 to 2021-04-01
Number of records is 783
done
